In [ ]:
import altair as alt
import pandas as pd

# https://altair-viz.github.io/gallery/simple_bar_chart.html
source = pd.DataFrame({
    'a': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'],
    'b': [28, 55, 43, 91, 81, 53, 19, 87, 52]
})

alt.Chart(source).mark_bar().encode(
    x='a',
    y='b'
)

In [ ]:
import pandas as pd
pd.set_option("display.max_rows", None)

In [ ]:
# inheriting usage notes from my other code for now (not self-plagiarization)
# https://github.com/cephcyn/cse517project/blob/master/embed_w2v.py
# for word2vec code; see also word2vec documentation
import gensim

# Load Google's pre-trained Word2Vec model.
# model source: https://code.google.com/archive/p/word2vec/
model = gensim.models.KeyedVectors.load_word2vec_format('model/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
def get_phrase_vector(phrase):
    try:
        phrase = phrase.split()
    except:
        return pd.DataFrame()
    unknowns = []
    emb = []
    for w in phrase:
        try:
            emb.append(model[w])
        except:
            unknowns.append(w)
    emb_sum = pd.DataFrame(emb).sum()
    return pd.DataFrame([emb_sum])

v = get_phrase_vector('hi')
v

In [ ]:
search_word = 'BERT'
anchor_type = 'coreference'

csv = pd.read_csv(f'outputs/{search_word}/{anchor_type}.csv')
#csv

In [ ]:
output = csv.groupby(csv.index, group_keys=False).apply(
    lambda group: get_phrase_vector(group.iloc[0]['averb'])
).reset_index(drop=True)
output

In [ ]:
# Visualize the vectors
# loaned from :
# https://github.com/cephcyn/ChatlogGrapher/blob/master/data_processing.ipynb

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import altair as alt
alt.renderers.enable('default')

def visualize_embeds(data):
    x = data
    x = StandardScaler().fit_transform(x)

    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data=principalComponents,
                               columns = ['pc1', 'pc2'])

    finalDf = principalDf
#     finalDf = pd.concat([principalDf, 
#                          pd.DataFrame(sen_top_bow.keys()).rename(columns={0:'name'})],
#                         axis = 1)

    return alt.Chart(finalDf).mark_circle(size=60).encode(
        x='pc1',
        y='pc2',
#         color='type',
#         tooltip=['name']
    ).interactive()

visualize_embeds(output)